# o-viton
## shape generation
### test

#### 필요한 패키지 설치

In [1]:
import os
import sys

IS_COLAB = False

if 'COLAB_GPU' in os.environ:
    IS_COLAB=True
else:
    IS_COLAB=False
 

In [2]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
!pip install dominate
!pip install av


# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

     |████████████████████████████████| 37.2MB 151kB/s 
     |████████████████████████████████| 276kB 13.4MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44075 sha256=cfa1795d295090b77f4148784dd90ebe51348f3d0c1d5ba595939d2d11c59a6b
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
1.7.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
# assert torch.__version__.startswith("1.7")
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [5]:
# console에서 다음 작업 진행할 것.

#!git clone https://github.com/wonyangcho/detectron2.git
!git clone https://github.com/wonyangcho/Image-Based-Virtual-Try-on-Network-from-Unpaired-Data.git o-viton

#git checkout develop

Cloning into 'o-viton'...
remote: Enumerating objects: 644, done.
remote: Counting objects: 100% (644/644), done.
remote: Compressing objects: 100% (492/492), done.
remote: Total 1381 (delta 334), reused 423 (delta 141), pack-reused 737
Receiving objects: 100% (1381/1381), 10.56 MiB | 11.86 MiB/s, done.
Resolving deltas: 100% (715/715), done.


In [6]:
if IS_COLAB == True:
    from google.colab import drive


In [7]:
!nvidia-smi

Sat Feb  6 04:45:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### 데이터 불러오기

In [8]:
from pathlib import Path

my_file = Path("/content/dataset")
if my_file.is_dir() == False:
  !wget -O o-viton-dataset.zip https://www.dropbox.com/s/exjzimmtfx6g4be/o-viton-dataset_test.zip?dl=0
  #!wget -O o-viton-dataset.zip https://www.dropbox.com/s/w1fm887fr56xm3d/dataset_test_org.zip?dl=0
  


  !unzip -o -q o-viton-dataset.zip 
  !rm o-viton-dataset.zip
!ls ./dataset


--2021-02-06 04:45:08--  https://www.dropbox.com/s/exjzimmtfx6g4be/o-viton-dataset_test.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/exjzimmtfx6g4be/o-viton-dataset_test.zip [following]
--2021-02-06 04:45:08--  https://www.dropbox.com/s/raw/exjzimmtfx6g4be/o-viton-dataset_test.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4a021e21e860d82415a426d608.dl.dropboxusercontent.com/cd/0/inline/BIbuScMHBbdy1Uwrf_w_jpcrUaDOhNxhcra47KqimrGKIK5eorHiz6ZQ2-aREOL5KqqKJYOCyxwfYTQbiX3Dh69UIUE3ejgFwkQEtn8V8sCEC7ohIYAwKngf0SnJlBHSrF8/file# [following]
--2021-02-06 04:45:09--  https://uc4a021e21e860d82415a426d608.dl.dropboxusercontent.com/cd/0/inline/BIbuScMHBbdy1Uwrf_w_jpcrUaDOhNxhcra47KqimrGKIK5eorHiz6ZQ2-aREOL5Kqq

#### google drive 연결

In [9]:
!ls /content/drive/MyDrive


ls: cannot access '/content/drive/MyDrive': No such file or directory


In [10]:
if IS_COLAB == True:
    drive.mount('/content/drive')
    !ls "/content/drive/My Drive/02_o-viton/"

Mounted at /content/drive
source


#### path 설정

In [11]:
if IS_COLAB == True:
  sys.path.append("/content/drive/My Drive/02_o-viton/source/shape_generation")
  sys.path.append("/content/o-viton/shape_generation")

#### 관련 모듈 불러오기 

In [12]:
from options.test_options import TestOptions
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.test_options import TestOptions
from data.ov_test_dataset import TestDataset
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import pickle

%matplotlib inline

#### checkpoint 경로 확인

In [13]:
!ls /content/drive/MyDrive/02_o-viton/source/shape_generation/checkpoints

vfr


#### main

In [14]:
def main():
    opt = TestOptions().parse(save=False)

    
    opt.checkpoints_dir = "/content/drive/My Drive/02_o-viton/source/shape_generation/checkpoints"
    opt.results_dir = "/content/drive/My Drive/02_o-viton/source/shape_generation/results"
    opt.dataroot = "./dataset"
   


    augment = {}
    # augment['1'] = transforms.Compose(
    #     [transforms.ToTensor(),
    #      transforms.Normalize((0.5, ), (0.5, ))])  # change to [C, H, W]
    augment['1'] = transforms.Compose(
        [transforms.ToTensor()])  # change to [C, H, W]

    test_dataset = TestDataset(opt, augment)
    test_dataloader = DataLoader(test_dataset,
                                shuffle=False,
                                num_workers=int(opt.nThreads),
                                batch_size=opt.batchSize,
                                pin_memory=True)
    dataset_size = len(test_dataset)
    print('#testing images = %d' % dataset_size)

    for key in test_dataset[0].keys():
        try:
            x = test_dataset[0][key]
            print("name of the input and shape -- > ", key, x.shape)
            print("type,dtype,and min max -- >", type(x),
                  x.dtype, torch.min(x), torch.max(x))
        except Exception as e:
            print("name of the input -- > ", key, test_dataset[0][key])
        print('----------------')

    # Create and Load Model
    model = create_model(opt)

    # Initialize visualizer
    visualizer = Visualizer(opt)
    # create website
    web_dir = os.path.join(opt.results_dir, opt.name, '%s_%s' %
                          (opt.phase, opt.which_epoch))
    webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt.name, opt.phase, opt.which_epoch))
    for i, data in enumerate(test_dataloader):
        #if i >= opt.how_many:
        #    break
        query_ref_mixed, generated = model.inference_enc(data['query'], data['dense_map'],data['ref'], cloth_part= opt.cloth_part)
        visuals = OrderedDict([('query', util.tensor2label(data['query'][0], opt.label_nc)),
                              ('ref', util.tensor2label(data['ref'][0], opt.label_nc)),
                              ('query_ref_mixed', util.tensor2label(query_ref_mixed.data[0], opt.label_nc)),
                              ('synthesized_image', util.tensor2label(generated.data[0], opt.label_nc)),
                              ('synthesized_image_edgemap', util.tensor2edgemap(torch.softmax(generated.data[0], dim=0)))])
        img_path = data['query_path']
        print('process image... %s' % img_path)
        visualizer.save_images(webpage, visuals, img_path)

    webpage.save()
        

if __name__ == '__main__':
    # torch.multiprocessing.set_start_method('spawn')
    main()

------------ Options -------------
aspect_ratio: 1.0
batchSize: 1
checkpoints_dir: ./checkpoints
cloth_part: uppercloth
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: /home/dataset/
densepose_nc: 27
display_winsize: 512
engine: None
export_onnx: None
feat_num: 10
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
input_nc: 20
instance_feat: False
isTrain: False
label_feat: False
label_nc: 20
lambda_feat: 10.0
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: ov_pix2pixHD
nThreads: 2
n_blocks_global: 9
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 4
n_layers_D: 3
n_local_enhancers: 1
name: vfr
ndf: 64
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_ce_loss: False
no_flip: False
no_ganFeat_loss: False
no_instance: False
no_lsgan: False
no_vgg_loss: False
norm: instance
ntest: inf
num_D: 2
onnx: None
output_nc: 20
phase: test
pool_size: 0
resize_or_crop: scale_width
results_dir: ./results/
serial_batches: False
